In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from random import randint
from time import sleep
import datetime

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

In [3]:
reviews = []

def extract_text(item, selector):
    try:
      return item.find(selector).text
    except AttributeError:
      return None

def getReviews(page):

    url = f'https://www.trustpilot.com/review/www.godaddy.com?page={page}'
    r = rq.get(url, headers=headers)
    soup = bs(r.text, 'html.parser')

    company_review = soup.findAll('div', attrs = {'class':'styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ'}) 

    for item in company_review:

        tag_reviews = 'p', {'class': 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'}
        tag_title = 'h2',{'class': 'typography_heading-s__f7029 typography_appearance-default__AAY17' }

        columns = {
        'title':extract_text(item,tag_title) ,
        'reviews':extract_text(item,tag_reviews),
        'location': item.find('div', {'class': 'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}).text,
        'ratings': str(item.find_all('img', alt = True)),
        'datetime': item.time.attrs['datetime'],
        
        }
        
        reviews.append(columns)
        
    return


In [5]:
for x in range(1,2515):
    print('Downloading page number', x)
    getReviews(x)
    sleep(randint(3, 5))

df = pd.DataFrame(reviews)


In [6]:
df

,title,reviews,location,ratings,datetime
0,Terrible response time,I appreciated the info but did not appreciate ...,CA,"[<img alt=""Rated 3 out of 5 stars"" src=""https:...",2022-12-09T01:04:35.000Z
1,Excellent From the Onset,"From the onset of the creation of my domains, ...",US,"[<img alt=""Rated 5 out of 5 stars"" src=""https:...",2022-12-08T17:15:56.000Z
2,On hold for over an hour…,"The GoDaddy representative that I spoke with, ...",US,"[<img alt=""Rated 4 out of 5 stars"" src=""https:...",2022-12-08T21:57:25.000Z
3,Customer service was great,Customer service was great. The only thing tha...,US,"[<img alt=""Rated 5 out of 5 stars"" src=""https:...",2022-12-08T19:51:34.000Z
4,Wait time TOOOOOOOO LONG!!!,I had to call Go Daddy twice today to migrate ...,US,"[<img alt=""Rated 3 out of 5 stars"" src=""https:...",2022-12-08T21:41:15.000Z
...,...,...,...,...,...
50295,thomas is an rockstar,thomas is an rockstar\r when i called go daddy...,US,"[<img alt=""Rated 5 out of 5 stars"" src=""https:...",2014-05-16T16:33:41.000Z
50296,I would recommend this company to anyone.,This is customer service\rby MUrsuy Media Crea...,US,"[<img alt="""" aria-hidden=""true"" src=""data:imag...",2014-05-16T13:42:17.000Z
50297,Costumer support is NO support,After I bought the domain I've waited a couple...,DK,"[<img alt="""" aria-hidden=""true"" src=""data:imag...",2014-10-17T04:40:55.000Z
50298,So far so good,Bought a domain and WordPress ability and it s...,DK,"[<img alt="""" aria-hidden=""true"" src=""data:imag...",2014-05-02T22:14:58.000Z


In [7]:
df.to_excel('raw_reviews.xlsx', index=False)

In [12]:
train = df.copy()

### Cleanings Ratings

In [13]:
try:
    train['ratings'] = train['ratings'].str.extract('Rated (.+?) out of', expand=False).str.strip()
except AttributeError:
    train['ratings'] = "none"

### Converting Datetime to Date and Time

In [14]:
def time_convert(date_to_convert):
     return datetime.datetime.strptime(date_to_convert, '%Y-%m-%dT%H:%M:%S.%f%z').strftime('%H:%M:%S')

def date(date_to_convert):
     return datetime.datetime.strptime(date_to_convert, '%Y-%m-%dT%H:%M:%S.%f%z').strftime('%d/%m/%Y')

train['time'] = train['datetime'].apply(time_convert)
train['date'] = train['datetime'].apply(date)
train.drop(['datetime'], axis=1, inplace=True)
train

,title,reviews,location,ratings,time,date
0,Terrible response time,I appreciated the info but did not appreciate ...,CA,3,01:04:35,09/12/2022
1,Excellent From the Onset,"From the onset of the creation of my domains, ...",US,5,17:15:56,08/12/2022
2,On hold for over an hour…,"The GoDaddy representative that I spoke with, ...",US,4,21:57:25,08/12/2022
3,Customer service was great,Customer service was great. The only thing tha...,US,5,19:51:34,08/12/2022
4,Wait time TOOOOOOOO LONG!!!,I had to call Go Daddy twice today to migrate ...,US,3,21:41:15,08/12/2022
...,...,...,...,...,...,...
50295,thomas is an rockstar,thomas is an rockstar\r when i called go daddy...,US,5,16:33:41,16/05/2014
50296,I would recommend this company to anyone.,This is customer service\rby MUrsuy Media Crea...,US,5,13:42:17,16/05/2014
50297,Costumer support is NO support,After I bought the domain I've waited a couple...,DK,2,04:40:55,17/10/2014
50298,So far so good,Bought a domain and WordPress ability and it s...,DK,4,22:14:58,02/05/2014


In [15]:
train.to_excel('cleaned_reviews.xlsx', index=False)